Stress test:
- There are 5531452 rows in the csv file in total (including header)
- It takes 3m31s to read train_data.csv

In [41]:
# N = sum(1 for line in open('data/train_data.csv'))
# N
# print(sum(1 for line in open('data/train_labels.csv')))

458914


In [39]:
# df = pd.read_csv('data/train_data.csv')
# labels = pd.read_csv('data/train_labels.csv')

## Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
x_test = pd.read_csv('data/train_data.csv', nrows=5)
y_test = pd.read_csv('data/train_labels.csv', nrows=5)
print(x_test.index)
print(x_test['customer_ID'] == y_test['customer_ID'])

RangeIndex(start=0, stop=5, step=1)
0     True
1    False
2    False
3    False
4    False
Name: customer_ID, dtype: bool


In [2]:
features = pd.read_csv('data/train_data.csv')
labels = pd.read_csv('data/train_labels.csv')

In [3]:
features.index = features['customer_ID']
labels.index = labels['customer_ID']
# labeled_features = l1.join(f1, how='inner')
id = features['customer_ID'][2]
print(id)
print(features['D_39'][2])
print(features.loc[features.index == id]['D_39'])

0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a
0.0915053967544593
customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.001733
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.005775
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.091505
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.002455
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.002483
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.001746
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.002183
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.003029
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.009896
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.001082
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.005715
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed79

There are multiple values of the same feature for the same customer

In [4]:
features.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

In [5]:
obj_cols = features.select_dtypes(include=['O']).columns
obj_cols

Index(['customer_ID', 'S_2', 'D_63', 'D_64'], dtype='object')

In [6]:
int_cols = features.select_dtypes(include=['int64']).columns
int_cols

Index(['B_31'], dtype='object')

In [7]:
print('B_31', features.B_31.unique())
for col in obj_cols.drop(['customer_ID', 'S_2']):
    print(col, features[col].unique())

B_31 [1 0]
D_63 ['CR' 'CO' 'CL' 'XZ' 'XM' 'XL']
D_64 ['O' 'R' nan 'U' '-1']


S_2 is a date

In [8]:
float_cols = features.select_dtypes(include=['float64']).columns
float_cols

Index(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42',
       'D_43',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=185)

In [9]:
nullcols = []
for col in features:
    if features[col].isnull().values.any():
        nullcols.append(col)
len(nullcols)

122

## Preprocessing

#### Clean up and label data:

In [15]:
def mean_acc(series: pd.Series):
    return series.mean()
    
def clean_features(features: pd.DataFrame, acc):
    features.index = features['customer_ID']
    df = pd.DataFrame(index=features.index.unique())
    # either make series then assign to dataframe or assign each entry individually
    for col in features:
        if features[col].dtype == 'float64':
            assigned_col = []
            for id in df.index:
                # print(features.loc[features.index == id][col])
                assigned_col.append(acc(features.loc[features.index == id][col]))
                print('1')
            df[col] = assigned_col
    return df
    # for id in df.index:
    #     for col in features:
    #         acc(features.loc[features.index == id][col])

cleaned_features = clean_features(features, mean_acc)
cleaned_features.head()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 